In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.common.keys import Keys
import chromedriver_binary
import configparser
import gspread
import time
import cv2
import numpy as np
from PIL import Image
import requests

from oauth2client.service_account import ServiceAccountCredentials 
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('suraping-381608-4562bacb42ee.json', scope)

def download_img(image, filename):
    r = requests.get(url=image, cookies=cookie, headers=headers, stream=True)
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(r.content)

date = 0 #当日は0、前日は1・・・
date2 = "08/19"
pmc = "21009013" # 店舗ID
SPREADSHEET_KEY = '1V-jIuti1-RCpPMMnuuCPF1MXMQGwJgE77oj60V96gB4' # スプレッドシートのID
pstart = 1 # パチンコの1台目
sstart = 661 # スロットの1台目
wait = 0.01

user_id = "tomosvtg.1988+789@gmail.com"
user_pw = "paruparu23"

Template_p_path = "site777_2/template_p.png"
Template_s_path = "site777_2/template_s.png"
dot_point_p = 130000 / 432
dot_point_px = 60000 / 234
dot_point_s = 20000 / 432

mobile_emulation = { "deviceName": "iPhone 12 Pro" }
headers = {
    "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 16_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/108.0.5359.112 Mobile/15E148 Safari/604.1",
    "Referer" : "https://m.site777.jp/f/D3000.do?pmc=22021007&gc=2&dn=5019&urt=2173&mdc=120104&dsgk=0&dtdd=2"
}

options = Options()
options.add_experimental_option("mobileEmulation", mobile_emulation)
#options.add_argument('--headless')
options.binary_location = 'C:\\Users\\user\\AppData\\Local\\Google\\Chrome\\Application\\chrome.exe'
config_ini = configparser.ConfigParser()
config_ini.read('config.ini', encoding='utf-8')

wd = config_ini['ChromeDriver']['WD']

path = "C:\\Users\\user\\WD\\" + str(wd) + "\\chromedriver.exe"
chrome_service = ChromeService(executable_path=path)

gc = gspread.authorize(credentials)
wb = gc.open_by_key(SPREADSHEET_KEY)

worksheet = wb.worksheet("Config")
p4row = worksheet.cell(11,  23).value
#p1row = worksheet.cell(11,  24).value
s20row = worksheet.cell(11,  28).value
#s5row = worksheet.cell(11,  29).value

browser = webdriver.Chrome(service=chrome_service,options=options)
browser.command_executor._commands["send_command"] = ("POST", '/session/$sessionId/chromium/send_command')
params = {'cmd': 'Page.setDownloadBehavior', 'params': {'behavior': 'allow', 'downloadPath': 'site777'}}
command_result = browser.execute("send_command", params)

browser.get('https://m.site777.jp/f/A0100.do')
time.sleep(1)

browser.find_element("xpath",'/html/body/section[1]/div/div/p[3]/a').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/section/section[2]/div/div/p').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[1]/input').send_keys(user_id)
browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[2]/input').send_keys(user_pw)
time.sleep(1)

browser.execute_script('document.fmLogin.submit()')
time.sleep(1)

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(pstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

pmdc = browser.current_url.split("mdc=")[1]

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(sstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

smdc = browser.current_url.split("mdc=")[1]

ck = browser.get_cookies()
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "PO_SP_ID":
        po_sp_id = ck[k]["value"]
        k = 30
    k = k + 1
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "JSESSIONID":
        jsessionid = ck[k]["value"]
        k = 30
    k = k + 1

cookie = {"PO_SP_ID" : po_sp_id,
         "JSESSIONID" : jsessionid}

browser.close()

p4table = []
#p1table = []
s20table = []
#s5table = []

# Pachinko

dn = pstart
mdc = pmdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D2600S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 100:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<hatariSu>")[m + cc].split("</hatariSu>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_p_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 341 or y > 343:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 342 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        game = round(dot_point_px * ( xx - 10) , 0)
                        samai = round(dot_point_p * point ,0)-300
                        if xx==10:
                            samai = 0
                            game = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass
                    
                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                print(li)
                if unit == "438":
                    p4table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                    print(li)
                    if unit == "438":
                        p4table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

# Slot

dn = sstart
mdc = smdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D3000S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 1000:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                game = jsonData.split("<totalGameCount>")[m + cc].split("</totalGameCount>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<regularCount>")[m + cc].split("</regularCount>")[0]
                art = jsonData.split("<artCount>")[m + cc].split("</artCount>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_s_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 332 or y > 334:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 333 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        samai = round(dot_point_s * point ,0)
                        if xx==10:
                            samai = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass

                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                print(li)
                if unit == "2173":
                    s20table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                    print(li)
                    if unit == "2173":
                        s20table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

wb.values_append("'P4RAW'!F" + str(int(p4row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p4table})
#wb.values_append("'P1RAW'!F" + str(int(p1row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p1table})
wb.values_append("'S20RAW'!F" + str(int(s20row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s20table})
#wb.values_append("'S5RAW'!F" + str(int(s5row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s5table})

['08/19', '025890', '1', '2564.0', '12339.0', '15', '1']
['08/19', '025890', '2', '0', '0', '0', '0']
['08/19', '025890', '3', '0', '0', '0', '0']
['08/19', '025890', '5', '0', '0', '0', '0']
['08/19', '025890', '6', '0', '0', '0', '0']
['08/19', '025890', '7', '0', '0', '0', '0']
['08/19', '025890', '8', '0', '0', '0', '0']
['08/19', '025890', '10', '2308.0', '-2106.0', '0', '0']
['08/19', '025890', '11', '1538.0', '-1504.0', '0', '0']
['08/19', '025888', '12', '1538.0', '-1203.0', '0', '0']
['08/19', '025888', '13', '11026.0', '-3309.0', '4', '1']
['08/19', '025888', '15', '15385.0', '-9930.0', '2', '2']
['08/19', '025888', '16', '7436.0', '10232.0', '11', '1']
['08/19', '025888', '17', '8718.0', '-3008.0', '3', '2']
['08/19', '025888', '18', '7179.0', '-3610.0', '2', '2']
['08/19', '025888', '20', '7949.0', '5719.0', '8', '2']
['08/19', '025888', '21', '4872.0', '-3008.0', '1', '1']
['08/19', '025888', '22', '6154.0', '7524.0', '8', '1']
['08/19', '025869', '23', '1795.0', '-1203.0'

['08/19', '025867', '223', '5128.0', '-2707.0', '1', '1']
['08/19', '025867', '225', '11795.0', '4515.0', '10', '3']
['08/19', '025867', '226', '8462.0', '6019.0', '11', '2']
['08/19', '025900', '227', '32821.0', '29793.0', '36', '7']
['08/19', '025900', '228', '39487.0', '-6018.0', '17', '4']
['08/19', '025900', '230', '34359.0', '2107.0', '19', '9']
['08/19', '025900', '231', '46154.0', '-16550.0', '14', '7']
['08/19', '025677', '232', '22821.0', '1806.0', '16', '2']
['08/19', '025677', '233', '18205.0', '-10231.0', '4', '3']
['08/19', '025677', '235', '18718.0', '-13240.0', '2', '1']
['08/19', '025677', '236', '24872.0', '-10531.0', '7', '3']
['08/19', '025677', '237', '27436.0', '15047.0', '29', '3']
['08/19', '025677', '238', '30513.0', '-15948.0', '8', '2']
['08/19', '025677', '250', '16154.0', '1205.0', '11', '1']
['08/19', '025677', '251', '12564.0', '-2406.0', '8', '1']
['08/19', '025677', '252', '28974.0', '-9629.0', '11', '4']
['08/19', '025579', '253', '33077.0', '-4513.0',

['08/19', '025665', '551', '21795.0', '12640.0', '21', '3']
['08/19', '025665', '552', '15128.0', '2107.0', '10', '1']
['08/19', '025665', '553', '19231.0', '-6018.0', '8', '3']
['08/19', '025665', '555', '14615.0', '-9328.0', '3', '3']
['08/19', '025665', '556', '28205.0', '-9629.0', '10', '2']
['08/19', '025665', '557', '25385.0', '6320.0', '19', '4']
['08/19', '025665', '558', '0', '0', '0', '0']
['08/19', '025665', '560', '22051.0', '-10531.0', '6', '2']
['08/19', '025665', '561', '10256.0', '-7221.0', '2', '2']
['08/19', '025665', '562', '27692.0', '24376.0', '32', '6']
['08/19', '025665', '563', '11795.0', '13242.0', '16', '3']
['08/19', '025665', '565', '6923.0', '3010.0', '6', '1']
['08/19', '025665', '566', '20513.0', '9931.0', '17', '2']
['08/19', '025665', '567', '20000.0', '-14744.0', '3', '3']
['08/19', '025665', '568', '17179.0', '17154.0', '20', '1']
['08/19', '025665', '570', '22821.0', '7223.0', '18', '5']
['08/19', '025665', '571', '10513.0', '-5717.0', '3', '3']
['08

['08/19', '120068', '768', '2917', '1111.0', '22', '13', '--']
['08/19', '120068', '770', '2770', '-417.0', '16', '5', '--']
['08/19', '120125', '771', '3037', '-972.0', '8', '6', '--']
['08/19', '120125', '772', '3760', '185.0', '16', '6', '--']
['08/19', '120125', '773', '5376', '-648.0', '18', '10', '--']
['08/19', '120125', '775', '2008', '-185.0', '7', '3', '--']
['08/19', '120125', '776', '3392', '-648.0', '11', '4', '--']
['08/19', '120125', '777', '5693', '-278.0', '21', '10', '--']
['08/19', '120125', '778', '3026', '-278.0', '11', '4', '--']
['08/19', '120125', '780', '4411', '-2361.0', '10', '6', '--']
['08/19', '120125', '781', '2442', '-1389.0', '6', '1', '--']
['08/19', '120125', '782', '7249', '926.0', '32', '14', '--']
['08/19', '120125', '783', '1723', '-648.0', '4', '5', '--']
['08/19', '120125', '785', '3291', '-463.0', '10', '8', '--']
['08/19', '120125', '786', '2819', '-602.0', '9', '3', '--']
['08/19', '120125', '787', '4464', '-556.0', '14', '10', '--']
['08/19'

['08/19', '120106', '1100', '56', '0', '0', '0', '--']
['08/19', '119916', '1101', '7558', '556.0', '29', '25', '--']
['08/19', '119916', '1102', '3066', '2454.0', '22', '10', '--']
['08/19', '119916', '1103', '987', '-1111.0', '0', '1', '--']
['08/19', '119916', '1105', '4689', '-2037.0', '9', '12', '--']
['08/19', '119916', '1106', '3433', '880.0', '18', '5', '--']
['08/19', '119916', '1107', '1815', '-509.0', '6', '3', '--']
['08/19', '119916', '1108', '3602', '-185.0', '13', '10', '--']
['08/19', '119916', '1110', '2586', '-509.0', '9', '4', '--']
['08/19', '119916', '1111', '1119', '-509.0', '2', '2', '--']
['08/19', '119916', '1112', '2628', '-1019.0', '5', '8', '--']
['08/19', '119916', '1113', '4266', '-694.0', '12', '16', '--']
['08/19', '119916', '1115', '5244', '231.0', '20', '16', '--']
['08/19', '119916', '1116', '6599', '1991.0', '30', '29', '--']
['08/19', '119916', '1117', '1194', '-556.0', '2', '4', '--']
['08/19', '119916', '1118', '4254', '-833.0', '13', '12', '--']


{'spreadsheetId': '1V-jIuti1-RCpPMMnuuCPF1MXMQGwJgE77oj60V96gB4',
 'updates': {'spreadsheetId': '1V-jIuti1-RCpPMMnuuCPF1MXMQGwJgE77oj60V96gB4',
  'updatedRange': 'S20RAW!F33338:M33609',
  'updatedRows': 272,
  'updatedColumns': 8,
  'updatedCells': 2176}}